In [2]:
%matplotlib notebook

import json
from collections import OrderedDict
import copy
import numpy as np
import matplotlib.pyplot as plt
from converters import SensorThings2Dict

In [3]:
with open("ABU1.txt") as f:
    data = []
    for line in f:
        features = SensorThings2Dict(line)
        data.append(list(features.values()))
    print(len(data))

5632


In [31]:
data = np.asarray(data)
#print(data[:,2:-1])
numerics = data[:,2:-1]
# for n in numerics:
#     plt.plot(n);
mask = np.random.rand(len(data)) < 0.7
train = data[mask]
test = data[~mask]
print("Train Total: {} Good: {} Faulty: {} Ratio: {}".format(len(train), len(train[train[:,-1]=='True']), len(train[train[:,-1]=='False']), float(len(train[train[:,-1]=='False']))/len(train)))
print("Test  Total: {} Good: {} Faulty: {} Ratio: {}".format(len(test), len(test[test[:,-1]=='True']), len(test[test[:,-1]=='False']), float(len(test[test[:,-1]=='False']))/len(train)))

Train Total: 3934 Good: 3718 Faulty: 216 Ratio: 0.0549059481444
Test  Total: 1698 Good: 1601 Faulty: 97 Ratio: 0.0246568378241


In [32]:
faulty = train[train[:,-1]=='False']
not_faulty = train[train[:,-1]=='True']
train = np.concatenate((not_faulty, np.repeat(faulty, 10, axis=0))) # repeate faulties
#train = np.concatenate((not_faulty[:len(faulty)], faulty))
print("Train Total: {} Good: {} Faulty: {} Ratio: {}".format(len(train), len(train[train[:,-1]=='True']), len(train[train[:,-1]=='False']), float(len(train[train[:,-1]=='False']))/len(train)))
train = train[np.random.permutation(train.shape[0])] # shuffle data

Train Total: 5878 Good: 3718 Faulty: 2160 Ratio: 0.367471929228
(5878L, 54L)


In [ ]:
train_data = train[:,2:-1].astype(np.float32)
train_labels = np.array(train[:,-1]=='False').astype(np.int32)
test_data = test[:,2:-1].astype(np.float32)
test_labels = np.array(test[:,-1]=='False').astype(np.int32)

In [38]:
def eval_metrics(expected, predicted):
    print("------------------------- EVALUATION -------------------------")
    print("Accuracy Score: {}".format(metrics.accuracy_score(expected, predicted)))
    print(metrics.classification_report(expected, predicted))
    print("Confusion Matrix:")
    print(metrics.confusion_matrix(expected, predicted))
    print("Kappa Score: {}".format(metrics.cohen_kappa_score(expected, predicted)))
    print("Matthews Correlation Coefficient: {}".format(metrics.matthews_corrcoef(expected, predicted)))
    print("--------------------------------------------------------------")

In [39]:
# CART Decision Tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

clf = DecisionTreeClassifier()
clf.fit(train_data, train_labels)

p = clf.predict(train_data)
pt = clf.predict(test_data)

eval_metrics(train_labels, p)
eval_metrics(test_labels, pt)

------------------------- EVALUATION -------------------------
Accuracy Score: 1.0
             precision    recall  f1-score   support

      False       1.00      1.00      1.00      2160
       True       1.00      1.00      1.00      3718

avg / total       1.00      1.00      1.00      5878

Confusion Matrix:
[[2160    0]
 [   0 3718]]
Kappa Score: 1.0
Matthews Correlation Coefficient: 1.0
--------------------------------------------------------------
------------------------- EVALUATION -------------------------
Accuracy Score: 0.931684334511
             precision    recall  f1-score   support

      False       0.40      0.38      0.39        97
       True       0.96      0.97      0.96      1601

avg / total       0.93      0.93      0.93      1698

Confusion Matrix:
[[  37   60]
 [  56 1545]]
Kappa Score: 0.353308512105
Matthews Correlation Coefficient: 0.353396390964
--------------------------------------------------------------


In [34]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

clf = RandomForestClassifier(n_estimators=100)
clf = clf.fit(train_data, train_labels)

p = clf.predict(train_data)
pt = clf.predict(test_data)

eval_metrics(train_labels, p)
eval_metrics(test_labels, pt)

------------------------- EVALUATION -------------------------
Accuracy Score: 1.0
             precision    recall  f1-score   support

      False       1.00      1.00      1.00      2160
       True       1.00      1.00      1.00      3718

avg / total       1.00      1.00      1.00      5878

Confusion Matrix:
[[2160    0]
 [   0 3718]]
Kappa Score: 1.0
Matthews Correlation Coefficient: 1.0
--------------------------------------------------------------
------------------------- EVALUATION -------------------------
Accuracy Score: 0.945229681979
             precision    recall  f1-score   support

      False       0.75      0.06      0.11        97
       True       0.95      1.00      0.97      1601

avg / total       0.93      0.95      0.92      1698

Confusion Matrix:
[[   6   91]
 [   2 1599]]
Kappa Score: 0.106507938304
Matthews Correlation Coefficient: 0.205405486469
--------------------------------------------------------------


In [35]:
# MLP
from sklearn.neural_network import MLPClassifier
from sklearn import metrics

clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(25,50,25), random_state=1, 
                    max_iter=1000, verbose=True, tol=1e-6)
clf = clf.fit(train_data, train_labels)

p = clf.predict(train_data)
pt = clf.predict(test_data)

eval_metrics(train_labels, p)
eval_metrics(test_labels, pt)

------------------------- EVALUATION -------------------------
Accuracy Score: 0.647329023477
             precision    recall  f1-score   support

      False       0.59      0.13      0.22      2160
       True       0.65      0.95      0.77      3718

avg / total       0.63      0.65      0.57      5878

Confusion Matrix:
[[ 290 1870]
 [ 203 3515]]
Kappa Score: 0.0950256577
Matthews Correlation Coefficient: 0.138549711009
--------------------------------------------------------------
------------------------- EVALUATION -------------------------
Accuracy Score: 0.891637220259
             precision    recall  f1-score   support

      False       0.07      0.07      0.07        97
       True       0.94      0.94      0.94      1601

avg / total       0.89      0.89      0.89      1698

Confusion Matrix:
[[   7   90]
 [  94 1507]]
Kappa Score: 0.0131960456082
Matthews Correlation Coefficient: 0.0131990830631
--------------------------------------------------------------


In [36]:
# Support Vector Machine
from sklearn.svm import SVC
from sklearn import metrics

clf = SVC()
clf.fit(train_data, train_labels)

p = clf.predict(train_data)
pt = clf.predict(test_data)

eval_metrics(train_labels, p)
eval_metrics(test_labels, pt)

------------------------- EVALUATION -------------------------
Accuracy Score: 0.982647158898
             precision    recall  f1-score   support

      False       0.96      1.00      0.98      2160
       True       1.00      0.98      0.99      3718

avg / total       0.98      0.98      0.98      5878

Confusion Matrix:
[[2150   10]
 [  92 3626]]
Kappa Score: 0.96296640369
Matthews Correlation Coefficient: 0.963393463897
--------------------------------------------------------------
------------------------- EVALUATION -------------------------
Accuracy Score: 0.901060070671
             precision    recall  f1-score   support

      False       0.05      0.04      0.05        97
       True       0.94      0.95      0.95      1601

avg / total       0.89      0.90      0.90      1698

Confusion Matrix:
[[   4   93]
 [  75 1526]]
Kappa Score: -0.00614414401704
Matthews Correlation Coefficient: -0.0061801590477
--------------------------------------------------------------


In [20]:
expected = [1,1,1,0,0]
predicted = [1,1,1,1,0]
print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

             precision    recall  f1-score   support

          0       1.00      0.50      0.67         2
          1       0.75      1.00      0.86         3

avg / total       0.85      0.80      0.78         5

[[1 1]
 [0 3]]
